<a href="https://colab.research.google.com/github/SabrineRiahi/Key_Localisation_FloydHub/blob/master/Darknet21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Darknet

In [0]:
!git clone https://github.com/pjreddie/darknet
%cd ./darknet
!make

# Training yolo on VOC


## Downloading datas

In [0]:
%cd /content/darknet/data
!wget https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
!wget https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
!wget https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
!tar xf VOCtrainval_11-May-2012.tar
!tar xf VOCtrainval_06-Nov-2007.tar
!tar xf VOCtest_06-Nov-2007.tar

In [0]:
# Generate labels
%cd /content/darknet/data
!wget https://pjreddie.com/media/files/voc_label.py
!python voc_label.py

## Adding key's data 

In [0]:
# Adding key's images
%cd /content
!git clone https://github.com/belarbi2733/keras_yolov3

%cd /content/keras_yolov3
!wget https://github.com/belarbi2733/keras_yolov3/releases/download/1/key_wb.zip
!wget https://github.com/belarbi2733/keras_yolov3/releases/download/1/bckgrnd.zip
!unzip bckgrnd.zip
!unzip key_wb.zip
!python keys_with_background.py --keys "key_wb" --background "bckgrnd" --output "/content/darknet/data/VOCdevkit/VOC2007/img"

In [0]:
# Rename annotations.csv file into txt file
%cd /content/darknet/data/VOCdevkit/VOC2007/img
import os
pre, ext = os.path.splitext("/content/darknet/data/VOCdevkit/VOC2007/img/annotations.csv")
os.rename("annotations.csv", pre + ".txt")

/content/darknet/data/VOCdevkit/VOC2007/img


In [0]:
# Generate 1 file "annotation.txt" for each image in the right directory
%cd /content/darknet/data/VOCdevkit/VOC2007/img
!wget https://raw.githubusercontent.com/SabrineRiahi/Key_Localisation_FloydHub/master/annotation_pythonv3.py
!python annotation_pythonv3.py

In [0]:
# Generate annotation_train.txt and annotation_test.txt == Directories of file img of keys
%cd /content/darknet/data/VOCdevkit/VOC2007/img
!wget https://raw.githubusercontent.com/SabrineRiahi/Key_Localisation_FloydHub/master/annotation_train_test.py
!python annotation_train_test.py

In [0]:
# Concatenate annotation_train.txt + 2007_train.txt = train.txt
filenames = ['/content/darknet/data/VOCdevkit/VOC2007/img/annotation_train.txt', '/content/darknet/data/2007_train.txt']
with open('/content/darknet/data/train.txt', 'w') as outfile : # Path to output file
    for fname in filenames :
        with open(fname) as infile :
            for line in infile :
                outfile.write(line)
# Concatenate annotation_test.txt + 2007_test.txt = test.txt
filenames = ['/content/darknet/data/VOCdevkit/VOC2007/img/annotation_test.txt', '/content/darknet/data/2007_test.txt']
with open('/content/darknet/data/test.txt', 'w') as outfile : # Path to output file
    for fname in filenames :
        with open(fname) as infile :
            for line in infile :
                outfile.write(line)
!mv /content/darknet/data/VOCdevkit/VOC2007/img/train.txt /content/darknet/data
!mv /content/darknet/data/VOCdevkit/VOC2007/img/test.txt /content/darknet/data

## REM : Modify files .voc .names .cfg

In [0]:
# modifier .cfg : nb classes, nb filtes
# modifier .names : 81 = clés
# modifier .voc : paths
# modifier .txt : path of img data test/train

## Downloading pretrained convolutionnal  weights

In [0]:
%cd /content/darknet
!wget https://pjreddie.com/media/files/darknet53.conv.74

## Training model

In [0]:
# voc.names -> data
%cd /content/darknet/data
!rm voc.names
!wget https://raw.githubusercontent.com/SabrineRiahi/Key_Localisation_FloydHub/master/voc.names
# voc.data, yolov3.cfg -> cfg
%cd /content/darknet/cfg
!rm voc.data
!rm yolov3.cfg
!wget https://raw.githubusercontent.com/SabrineRiahi/Key_Localisation_FloydHub/master/voc.data
!wget https://raw.githubusercontent.com/SabrineRiahi/Key_Localisation_FloydHub/master/yolov3.cfg
# Makefile
%cd /content/darknet
!rm Makefile
!wget https://raw.githubusercontent.com/SabrineRiahi/Key_Localisation_FloydHub/master/Makefile

In [0]:
%cd /content/darknet
!./darknet detector train cfg/voc.data cfg/yolov3.cfg darknet53.conv.74 

## Testing model

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

%cd /content
## Google Authentification
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Download weights file
## Lien : https://drive.google.com/open?id=1W1BfADa1zQo9Ydzi3-686NitLceU_qgI
downloaded = drive.CreateFile({'id': '1W1BfADa1zQo9Ydzi3-686NitLceU_qgI'})
downloaded.GetContentFile('yolov3.backup')

## Download weights file
## Lien : https://drive.google.com/open?id=1CUDmf7NLvNSpeDS1E4KWLNW1CV7L8wWe
downloaded = drive.CreateFile({'id': '1CUDmf7NLvNSpeDS1E4KWLNW1CV7L8wWe'})
downloaded.GetContentFile('yolov3_900.weights')

/content


In [0]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

def download(path):
  from google.colab import files
  files.download(path)

%cd /content
!wget https://raw.githubusercontent.com/SabrineRiahi/Key_Localisation_FloydHub/master/yolov3-test.cfg

%cd /content/darknet
!./darknet detector test cfg/voc.data /content/yolov3-test.cfg /content/yolov3.backup /content/index.jpg -thresh 0.01
imShow('predictions.jpg')

In [0]:
### Optionnal
## Create result.txt file with prediction of some images

## Before : create new images 
#!./darknet detector test cfg/voc.data /content/yolov3-test.cfg /content/yolov3_900.weights < /content/darknet/data/VOCdevkit/VOC2007/img/annotation_test.txt > /content/result.txt
#filenames = ['/content/darknet/data/train.txt', '/content/darknet/data/test.txt']
#with open('/content/darknet/data/allimages.txt', 'w') as outfile : # Path to output file
#    for fname in filenames :
#        with open(fname) as infile :
#            for line in infile :
#                outfile.write(line)

## Real time detection on a Video

In [0]:
!./darknet detector demo cfg/voc.data /content/yolov3-test.cfg /content/yolov3_900.weights <video-file>